In [ ]:
# install youtubedl library
# sudo -H pip install --upgrade youtube-dl

In [ ]:
import os

%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

## download youtube video

# Detecting emotions

In [1]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import time
import os

%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


In [2]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = '33d85b37f8474e40b649999562336739'  ######### Here you have to paste your primary key for EMOTION API
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d9f4de948e934d4d8bbf101565c2697a') ######### set the key for FACE API 

time: 2.48 ms


In [3]:
# Image path
urlImage= ''

# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 36.9 ms


In [4]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 7.54 ms


In [17]:
# generate a fake url list to replicate S3 condition

#### ATTNTION ----- change '4641' to match the number of last image+1 -----#######
#### ATTNTION ----- change '1_23_17/1_23_17___%d.jpg' to match the path and names on your S3 bucket -----#######

url_list = []
for i in range(0,2179,1):
    url = "https://s3.amazonaws.com/100daysofspice/4_11_17/4_11_17___%d.jpg" % i
    url_list.append(url)

time: 4.6 ms


In [18]:
# Face Detection & Emotion Analysis - UPDATED
processed = {}
face_detection = {}
count = 0

for single_image in url_list:
 
    try:
        face_detection[count] = detect(single_image)

        json = { 'url': single_image } 
        processed[count] = processRequest(json, data, headers, params)  
    except:
        pass

    try:
        if face_detection[count][0]['faceRectangle'] == processed[count][0]['faceRectangle']:
            face_detection[count][0].update(processed[count][0])
            face_detection[count][1].update(processed[count][1])
        if  face_detection[count][0]['faceRectangle'] == processed[count][1]['faceRectangle']:
            face_detection[count][0].update(processed[count][1])
            face_detection[count][1].update(processed[count][0])
    except:
        pass
    
    count += 1
    
#     time.sleep(3)

status_code: 400
response: {"error":{"code":"InvalidURL","message":"Invalid image URL or error downloading from target server. Remote server error returned: \"Response status code does not indicate success: 404 (Not Found).\""}}
status_code: 400
response: {"error":{"code":"InvalidURL","message":"Invalid image URL or error downloading from target server. Remote server error returned: \"Response status code does not indicate success: 404 (Not Found).\""}}
status_code: 400
response: {"error":{"code":"InvalidURL","message":"Invalid image URL or error downloading from target server. Remote server error returned: \"Response status code does not indicate success: 404 (Not Found).\""}}
status_code: 400
response: {"error":{"code":"InvalidURL","message":"Invalid image URL or error downloading from target server. Remote server error returned: \"Response status code does not indicate success: 404 (Not Found).\""}}
status_code: 400
response: {"error":{"code":"InvalidURL","message":"Invalid image UR

In [8]:
face_detection

{0: [{'faceAttributes': {'gender': 'male'},
   'faceId': '75845ed1-503e-4599-b484-0af0ff62758a',
   'faceRectangle': {'height': 66, 'left': 252, 'top': 126, 'width': 66},
   'scores': {'anger': 0.000390850066,
    'contempt': 8.290622e-06,
    'disgust': 1.46704342e-05,
    'fear': 3.82832383e-07,
    'happiness': 0.999037743,
    'neutral': 0.000402712,
    'sadness': 0.000141318029,
    'surprise': 4.04186767e-06}}],
 1: [{'faceAttributes': {'gender': 'male'},
   'faceId': '4fe3d77b-3ab6-46ae-abf4-60f3a37c8d18',
   'faceRectangle': {'height': 70, 'left': 252, 'top': 113, 'width': 70},
   'scores': {'anger': 0.00121343171,
    'contempt': 0.00411117636,
    'disgust': 0.0009797838,
    'fear': 5.71572855e-05,
    'happiness': 0.570847869,
    'neutral': 0.40458256,
    'sadness': 0.00391385751,
    'surprise': 0.014294154}}],
 2: [{'faceAttributes': {'gender': 'male'},
   'faceId': '37c970bd-20ac-4441-9b83-45704754eae2',
   'faceRectangle': {'height': 72, 'left': 244, 'top': 114, 'wid

time: 633 ms


In [19]:
# Save the result as Json
#### ATTNTION ----- update '1_23_17.json' to the name of your current folder #######

import json
with open('/Users/Amiros/GitHub/100_days_of_spice/json/4_11_17.json', 'w') as fp:
    json.dump(face_detection, fp, sort_keys=True)

time: 169 ms


## json to csv

In [10]:
import json
import csv

time: 4.57 ms


In [11]:
"""
This can probably be improved but serves the purpose of our analysis.
I used Chris's schema so it will be useful for Quarz analysis.

"""

EMOTIONS = [
    'contempt',
    'fear',
    'sadness',
    'disgust',
    'anger',
    'neutral',
    'happiness',
    'surprise',
]


CSV_COLUMNS = [
    'url',
    'frame_time',
    'sean_happiness',
    'sean_neutral',
    'sean_anger',
    'sean_contempt',
    'sean_suprise',
    'sean_sadness',
    'sean_fear'
]

time: 4.15 ms


In [20]:
with open('/Users/Amiros/GitHub/100_days_of_spice/json/4_11_17.json') as data_file:    
    json_data = json.load(data_file)
    
json_data = {int(k):v for k,v in json_data.items()} #converting keys to integer   
frame_keys = sorted(json_data.keys())

time: 21.7 ms


In [21]:
csv_data = []
   
for key in frame_keys:
    url = 'https://s3.amazonaws.com/100daysofspice/4_11_17/4_11_17___%d.jpg' % key
    frame = json_data[key]
        
    row = {
        #'frame': frame,
        'url': url,
        'frame_time': key,
    }
    
    if len(frame) > 1:
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'sean_'
                try:
                    for emotion in EMOTIONS:
                        row[prefix + emotion] = face['scores'][emotion]
                except:
                    pass
                
            row.pop("frame", None)        
      
    elif len(frame) == 1:
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'sean_'
                try:
                    for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                        row[prefix + emotion] = face['scores'][emotion]
                except:
                    pass
                ### assign emotions vale to each candidate and change the name to the respective one 
          
        row.pop("frame", None)        

    csv_data.append(row)

time: 35.2 ms


In [22]:
import pandas as pd
df = pd.DataFrame(csv_data)

time: 12.2 ms


In [15]:
df['data'] = '4_11_17'

time: 2.75 ms


In [23]:
df.to_csv('csv/4_11_17-EMOTIONS.csv', sep=',', index=False)

time: 44.7 ms
